## Setting up a SWAT Connection for Python

In [7]:
import os
import swat

In [8]:
os.environ['CAS_CLIENT_SSL_CA_LIST'] = R"C:\SAS Viya\CAS_ex_cert.crt"

In [9]:
conn = swat.CAS(
    hostname="sas-cas-server-default-bin-demo.uksouth.cloudapp.azure.com",
    port=5570,
    authinfo=R"C:\SAS Viya\authinfo"
)

In [10]:
conn.serverstatus()

NOTE: Grid node action status report: 1 nodes, 8 total actions executed.


[About]

 {'CAS': 'Cloud Analytic Services',
  'Version': '4.00',
  'VersionLong': 'V.04.00M0P09182023',
  'Viya Release': '20230928.1695876470805',
  'Viya Version': 'Stable 2023.09',
  'Copyright': 'Copyright © 2014-2023 SAS Institute Inc. All Rights Reserved.',
  'ServerTime': '2023-10-03T12:23:30Z',
  'System': {'Hostname': 'controller.sas-cas-server-default.sas-viya.svc.cluster.local',
   'OS Name': 'Linux',
   'OS Family': 'LIN X64',
   'OS Release': '5.15.0-1041-azure',
   'OS Version': '#48-Ubuntu SMP Tue Jun 20 20:34:08 UTC 2023',
   'Model Number': 'x86_64',
   'Linux Distribution': 'Red Hat Enterprise Linux release 8.8 (Ootpa)'},
  'license': {'site': 'AMADEUS SOFTWARE LTD VIYA 4',
   'siteNum': 70282179,
   'expires': '30 June 2024 00:00:00',
   'gracePeriod': 30,
   'warningPeriod': 30},
  'CASHostAccountRequired': 'OPTIONAL',
  'Transferred': 'NO',
  'CASCacheLocation': 'CAS Disk Cache'}

[server]

 Server Status
 
    nodes  actions
 0      1        8

[nodestatus]

 Node Status
 
                                                 name        role  uptime  running  stalled
 0  controller.sas-cas-server-default.sas-viya.svc...  controller   2.913        0        0

+ Elapsed: 0.00148s, user: 0.00113s, sys: 0.000296s, mem: 0.309mb

## Create a reference to in-memory tables

In [ ]:
cirrhosis = conn.CASTable("cirrhosis", caslib="public")

## Use pandas code and functions on the CAS tables as if they were standard dataframes

These will be translated into CAS actions, and alter the underlying table in CAS

In [ ]:
# Get head data from table, methods similar to pandas
cirrhosis.head()

In [ ]:
# Work with the tables as if they were pandas dataframes

cirrhosis["Bilirubin_high"] = cirrhosis["Bilirubin"] > 1.2

## Use the full capabilities of CAS

Can load in Action Sets to perform any CAS action, i.e. powerful in-memory & distributed analysis

In [ ]:
conn.loadactionset("regression")

In [ ]:
cirrhosis.glm(
    target = "Platelets",
    inputs = ["Stage", "Status", "Drug", "Bilirubin_high", "Albumin", "Copper"],
    nominals = ["Stage"],
    # display = {"names": "ParameterEstimates"},
    output = {
        "casOut": {"name": "PlateletsPrediction", "replace": True},
        "copyvars": "all",
        "pred": "PredictedPlatelets",
        "resid": "ResidualPlatelets",
        "lcl": "LCL_Plt",
        "ucl": "UCL_Plt"
    }
)


In [ ]:
result = conn.CASTable("PlateletsPrediction")

In [ ]:
result.head()

In [ ]:
result.plot.scatter("PredictedPlatelets", "Platelets")